<a href="https://www.kaggle.com/code/pes1ug22am164/image-part?scriptVersionId=171996997" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# uncomment on google colab, comment on kaggle.

In [1]:
# !mkdir ~/.kaggle
# from google.colab import files
# files.upload()
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d shusrith/betterer-preproc

In [2]:
# !unzip /content/betterer-preproc.zip

In [3]:
# !kaggle datasets download -d tourist55/alzheimers-dataset-4-class-of-images

In [4]:
# !unzip /content/alzheimers-dataset-4-class-of-images.zip

# PRE PROCESS
.JPG TO .NPY

In [5]:
# import os
# import cv2
# import numpy as np

# def jpg_to_npy_in_folders(input_folder, output_folder):
#     os.makedirs(output_folder, exist_ok=True)
#     for i, item in enumerate(os.listdir(input_folder)):
#         item_path = os.path.join(input_folder, item)
#         if os.path.isdir(item_path):
#             jpg_to_npy_in_folders(item_path, output_folder)
#         elif os.path.isfile(item_path) and item.endswith(".jpg"):
#             img = cv2.imread(item_path)
#             img_array = np.array(img)
#             output_file_path = os.path.join(output_folder, os.path.splitext(item)[0] + '.npy')
#             np.save(output_file_path, img_array)

# # input_folder_path = "/content/Alzheimer_s Dataset/train"
# input_folder_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
# output_folder_path = "/kaggle/working/train"

# jpg_to_npy_in_folders(input_folder_path, output_folder_path)



In [6]:
# import os
# import cv2
# import numpy as np

# def jpg_to_npy_in_folders(input_folder, output_folder):
#     os.makedirs(output_folder, exist_ok=True)
#     for i, item in enumerate(os.listdir(input_folder)):
#         item_path = os.path.join(input_folder, item)
#         if os.path.isdir(item_path):
#             jpg_to_npy_in_folders(item_path, output_folder)
#         elif os.path.isfile(item_path) and item.endswith(".jpg"):
#             img = cv2.imread(item_path)
#             img_array = np.array(img)
#             folder = input_folder.split("/")[-1]
#             output_file_path = os.path.join(output_folder, f"{folder}{i}.npy")
#             np.save(output_file_path, img_array)

# # input_folder_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train"
# # output_folder_path = "/kaggle/working/train"

# # jpg_to_npy_in_folders(input_folder_path, output_folder_path)
# jpg_to_npy_in_folders("/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/test", "/kaggle/working/test")
# # jpg_to_npy_in_folders("/content/Alzheimer_s Dataset/test", "/kaggle/working/test")



In [7]:
# import os
# import numpy as np
# directory = '/kaggle/working/train'

# files = [file for file in os.listdir(directory) if file.endswith('.npy')]
# len(files)


In [8]:
# mildDem = [i for i in files if i.startswith("mildDem")]
# moderateDem = [i for i in files if "moderateDem" in i]
# nonDem = [i for i in files if "nonDem" in i]
# veryMild = [i for i in files if "verymildDem" in i]
# print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

In [9]:
# import imgaug.augmenters as iaa

# # Define the augmenter
# augmenter = iaa.Sequential([
#     iaa.Affine(rotate=(-10, 10)),  # Rotate the image by -10 to 10 degrees
#     iaa.GaussianBlur(sigma=(0.0, 1.0)),  # Apply Gaussian blur with a sigma ranging from 0.0 to 1.0
#     iaa.AdditiveGaussianNoise(scale=(0, 0.05*255)),  # Add Gaussian noise with a scale of 0 to 0.05 times 255
#     iaa.Multiply((0.9, 1.1)),
#     iaa.LinearContrast((0.9, 1.1)),
# ])

# augmented_images = []

# for path in moderateDem:
#     path = os.path.join("/kaggle/working/train", path)
#     image = np.load(path)

#     for _ in range(35):
#         augmented_image = augmenter.augment_image(image)
#         augmented_images.append(augmented_image)
# augmented_images = np.array(augmented_images)

In [10]:
# len(augmented_images)

In [11]:
# import cv2
# import os


# output_directory = "/kaggle/working/train"

# os.makedirs(output_directory, exist_ok=True)

# for i, image in enumerate(augmented_images):
#     output_path = os.path.join(output_directory, f"moderateDem{i+53}.npy")
#     np.save(output_path, image)


# print("Augmented images saved successfully.")


In [12]:
# augmented_images = []

# for path in mildDem:
#     path = os.path.join("/kaggle/working/train", path)
#     image = np.load(path)

#     for _ in range(2):
#         augmented_image = augmenter.augment_image(image)
#         augmented_images.append(augmented_image)
# augmented_images = np.array(augmented_images)

In [13]:
# len(augmented_images)

In [14]:
# output_directory = "/kaggle/working/train"

# os.makedirs(output_directory, exist_ok=True)

# for i, image in enumerate(augmented_images):
#     output_path = os.path.join(output_directory, f"mildDem{i+718}.npy")
#     np.save(output_path, image)


# print("Augmented images saved successfully.")

In [15]:
# import os
# import numpy as np
# directory = '/kaggle/working/train'

# files = [file for file in os.listdir(directory) if file.endswith('.npy')]
# len(files)

In [16]:
# mildDem = [i for i in files if i.startswith("mildDem")]
# moderateDem = [i for i in files if "moderateDem" in i]
# nonDem = [i for i in files if "nonDem" in i]
# veryMild = [i for i in files if "verymildDem" in i]
# print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

In [17]:
# import pandas as pd
# l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
# x = []
# for i in l:
#     for j in i[0]:
#         path = os.path.join("/kaggle/working/train", j)
#         x.append([i[1], path])

# df = pd.DataFrame(x, columns=["Class", "Path"])
# x = df["Class"].value_counts()
# x

In [18]:
# images = []
# for i in df["Path"]:
#     images.append(np.load(i))
# images = np.array(images)

In [19]:
# from imblearn.over_sampling import SMOTE

# sm = SMOTE(random_state=42)
# flattened_images = [image.flatten() for image in images]
# image_shape = images[0].shape
# X_res, y_res = sm.fit_resample(flattened_images, np.array(df["Class"]))  # `labels` should be your list of class labels


In [20]:
# res_images = [np.array(x).reshape(image_shape) for x in X_res]

In [21]:
# !rm -rf /kaggle/working/train

In [22]:
# import os

# names = {0:"milDem", 1:"moderateDim", 2:"nonDem", 3:"veryMildDem"}

# # Create the directory if it doesn't exist
# os.makedirs("/kaggle/working/train/", exist_ok=True)

# for i in range(len(y_res)):
#     np.save(f"/kaggle/working/train/{names[y_res[i]]}{i}.npy", res_images[i])

In [23]:
# import pandas as pd
# labels = pd.DataFrame(y_res)
# labels.value_counts()

In [24]:
# images = np.array(res_images)

In [25]:
import os
import numpy as np
directory = '/kaggle/input/betterest-preproc/train'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

10240

In [26]:
mildDem = [i for i in files if i.startswith("milDem")]
moderateDem = [i for i in files if "moderateDim" in i]
nonDem = [i for i in files if "nonDem" in i]
veryMild = [i for i in files if "veryMildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

2560 2560 2560 2560


In [27]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/input/betterest-preproc/train", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
x = df["Class"].value_counts()
x

Class
0    2560
1    2560
2    2560
3    2560
Name: count, dtype: int64

In [28]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [29]:
from sklearn.model_selection import train_test_split
labels = np.array(df["Class"])
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


In [30]:
import gc

# Assume `arr` is your numpy array
del images

# Run the garbage collector
gc.collect()

0

In [31]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Verify the encoded labels
print(train_labels_encoded)
print(test_labels_encoded)

[0 2 0 ... 2 0 2]
[1 2 1 ... 1 3 2]


In [32]:
print(train_images.shape, test_images.shape, train_labels_encoded.shape, test_labels_encoded.shape)

(8192, 208, 176, 3) (2048, 208, 176, 3) (8192,) (2048,)


In [33]:
print(test_labels)
test_labels_encoded

[1 2 1 ... 1 3 2]


array([1, 2, 1, ..., 1, 3, 2])

In [34]:
# print(x.index, x.values, x.array)

In [35]:
# s = x / len(files)
# class_weights = {i:j for i, j in zip(s.index, s.values)}

# class_weights
# Optional normalization (weights sum to 1)
# total_weight = sum(class_weights.values())
# class_weights = {class_label: weight / total_weight for class_label, weight in class_weights.items()}


In [36]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
import numpy as np

def conv_block(filters, act='relu'):
    """Defining a Convolutional NN block for a Sequential CNN model. """

    block = Sequential()
    block.add(Conv2D(filters, 3, activation=act, padding='same'))
    block.add(Conv2D(filters, 3, activation=act, padding='same'))
    block.add(BatchNormalization())
    block.add(MaxPooling2D((2, 2)))

    return block

2024-04-18 01:21:41.517828: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 01:21:41.517975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 01:21:41.662440: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [37]:
def dense_block(units, dropout_rate, act='relu'):
    """Defining a Dense NN block for a Sequential CNN model. """

    block = Sequential()
    block.add(Dense(units, activation=act))
    block.add(BatchNormalization())
    block.add(Dropout(dropout_rate))

    return block

In [38]:
# !pip install tensorflow_addons


In [39]:
import tensorflow as tf

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes, **kwargs):
        super(F1Score, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

METRICS = [tf.keras.metrics.CategoricalAccuracy(name='acc'),
           tf.keras.metrics.AUC(name='auc'),
           F1Score(num_classes=4)]

In [40]:
input_shape = train_images[0].shape

train_labels_encoded = to_categorical(train_labels, num_classes=4)
test_labels_encoded = to_categorical(test_labels, num_classes=4)

# sample_weights = compute_sample_weight(class_weight='balanced', y=np.argmax(train_labels_encoded, axis=1))

In [41]:
def construct_model(act='relu'):
    """Constructing a Sequential CNN architecture for performing the classification task. """

    model = Sequential([
        Input(shape=(input_shape)),
        conv_block(16),
        conv_block(32),
        conv_block(64),
        conv_block(128),
        Dropout(0.2),
        conv_block(256),
        Dropout(0.2),
        Flatten(),
        dense_block(512, 0.7),
        dense_block(128, 0.5),
        dense_block(64, 0.3),
        Dense(4, activation='softmax')
    ], name = "cnn_model")

    return model

In [42]:
model = construct_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=METRICS)

model.summary()


Model: "cnn_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 104, 88, 16)    │         2,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 52, 44, 32)     │        14,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 26, 22, 64)     │        55,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (None, 13, 11, 128)    │       221,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 11, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (None, 6, 5, 256)      │       886,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 6, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 7680)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_5 (Sequential)       │ (None, 512)            │     3,934,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (None, 128)            │        66,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ (None, 64)             │         8,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,190,420 (19.80 MB)

 Trainable params: 5,188,020 (19.79 MB)

 Non-trainable params: 2,400 (9.38 KB)

In [43]:
from keras.optimizers import Adam
def lr_schedule(epoch, learning_rate):
    if epoch < 20:
        return learning_rate
    else:
        if epoch % 2 == 0:
          return learning_rate * 0.95
        else:
          return learning_rate

lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)

initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)

In [44]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('val_acc') > 0.995:
            print("\nReached accuracy threshold! Terminating training.")
            self.model.stop_training = True

my_callback = MyCallback()

CALLBACKS = [my_callback, lr_scheduler]

In [45]:
EPOCHS = 50

history = model.fit(train_images, train_labels_encoded, validation_data=(test_images, test_labels_encoded), callbacks=CALLBACKS, epochs=EPOCHS)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
  1/256 ━━━━━━━━━━━━━━━━━━━━ 2:33:55 36s/step - acc: 0.2500 - auc: 0.5129 - f1_score: 0.1154 - loss: 2.2188

I0000 00:00:1713403368.188167      71 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1713403368.226328      71 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


256/256 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - acc: 0.3953 - auc: 0.6659 - f1_score: 0.3581 - loss: 1.6310

W0000 00:00:1713403390.718335      71 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


256/256 ━━━━━━━━━━━━━━━━━━━━ 60s 93ms/step - acc: 0.3954 - auc: 0.6661 - f1_score: 0.3582 - loss: 1.6302 - val_acc: 0.4575 - val_auc: 0.7604 - val_f1_score: 0.4480 - val_loss: 1.4533 - learning_rate: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
256/256 ━━━━━━━━━━━━━━━━━━━━ 17s 68ms/step - acc: 0.4623 - auc: 0.7621 - f1_score: 0.4084 - loss: 1.1533 - val_acc: 0.4502 - val_auc: 0.7238 - val_f1_score: 0.3608 - val_loss: 1.7912 - learning_rate: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
256/256 ━━━━━━━━━━━━━━━━━━━━ 18s 68ms/step - acc: 0.4780 - auc: 0.7814 - f1_score: 0.4004 - loss: 1.0828 - val_acc: 0.5063 - val_auc: 0.8061 - val_f1_score: 0.3681 - val_loss: 0.9947 - learning_rate: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
256/256 ━━━━━━━━━━━━━━━━━━━━ 18s 69ms/step - acc: 0.4962 - auc: 0.8010 - f1_score: 0.3962 - loss: 1.0072 -

In [46]:


# # Assuming train_images and test_images are your input image data


# model = Sequential()
# model.add(Input(shape=input_shape))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4, activation='softmax'))


In [47]:
# import math
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# history = model.fit(train_images, train_labels_encoded,
#                     steps_per_epoch = int(len(train_images) / 32),
#                     epochs=50,
#                     validation_data=(test_images, test_labels_encoded),
#                     sample_weight=sample_weights,
#                     callbacks=[lr_scheduler])

In [48]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score

y_prob = model.predict(test_images)
y_prob = tf.nn.softmax(y_prob).numpy()

# Compute ROC AUC for each class
roc_auc = roc_auc_score(test_labels_encoded, y_prob, average='macro')

print("ROC AUC:", roc_auc)

10/64 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step

W0000 00:00:1713404288.107501      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step
ROC AUC: 0.9948394783364186


In [49]:
import os
import numpy as np
directory = '/kaggle/input/betterest-preproc/test'

files = [file for file in os.listdir(directory) if file.endswith('.npy')]
len(files)

1279

In [50]:
mildDem = [i for i in files if i.startswith("MildDem")]
moderateDem = [i for i in files if "ModerateDem" in i]
nonDem = [i for i in files if "NonDem" in i]
veryMild = [i for i in files if "VeryMildDem" in i]
print(len(mildDem), len(moderateDem), len(nonDem), len(veryMild))

179 12 640 448


In [51]:
import pandas as pd
l = [[mildDem, 0], [moderateDem, 1], [nonDem, 2], [veryMild, 3]]
x = []
for i in l:
    for j in i[0]:
        path = os.path.join("/kaggle/input/betterest-preproc/test", j)
        x.append([i[1], path])

df = pd.DataFrame(x, columns=["Class", "Path"])
df["Class"].value_counts()


Class
2    640
3    448
0    179
1     12
Name: count, dtype: int64

In [52]:
images = []
for i in df["Path"]:
    images.append(np.load(i))
images = np.array(images)

In [53]:
labels = df["Class"]
labels = to_categorical(labels, num_classes=4)
print(labels.shape)

(1279, 4)


In [54]:
l = []
for i in labels:
  for j in range(4):
    if i[j] == 1:
      l.append(j)


In [55]:
y_prob = model.predict(images)

# y_prob = tf.nn.softmax(y_prob).numpy()
# Get the class with the highest probability for each prediction
y_prob = np.argmax(y_prob, axis=1)  # Argmax gives index of max value
# for i in range(len(labels)):
#   print(l[i], y_prob[i])

12/40 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

W0000 00:00:1713404317.523468      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 142ms/step


W0000 00:00:1713404323.045520      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [56]:
from sklearn.metrics import classification_report

# Assuming you have predicted labels (predicted_classes) and true labels (test_labels_encoded)
report = classification_report(l, y_prob, output_dict=True)
for i in report:
  print(i, report[i])

0 {'precision': 0.6181818181818182, 'recall': 0.5698324022346368, 'f1-score': 0.5930232558139535, 'support': 179}
1 {'precision': 1.0, 'recall': 0.4166666666666667, 'f1-score': 0.5882352941176471, 'support': 12}
2 {'precision': 0.8808510638297873, 'recall': 0.646875, 'f1-score': 0.745945945945946, 'support': 640}
3 {'precision': 0.5758998435054773, 'recall': 0.8214285714285714, 'f1-score': 0.6770929162833487, 'support': 448}
accuracy 0.6950742767787333
macro avg {'precision': 0.7687331813792707, 'recall': 0.6137006600824688, 'f1-score': 0.6510743530402239, 'support': 1279}
weighted avg {'precision': 0.7383912089101353, 'recall': 0.6950742767787333, 'f1-score': 0.6989468477095035, 'support': 1279}


In [57]:
model.evaluate(images, labels)

10/40 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - acc: 0.5942 - auc: 0.8107 - f1_score: 0.5971 - loss: 2.0702

W0000 00:00:1713404333.174693      71 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - acc: 0.6305 - auc: 0.8253 - f1_score: 0.6318 - loss: 1.9542


W0000 00:00:1713404334.809564      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


[1.5857348442077637, 0.6950742602348328, 0.8619475960731506, 0.695924699306488]